# Test: the impact of SNR
---
What effect does the SNR have upon the recovery of the mass-weighted age of the stellar population? In this notebook, we will run our ppxf pipline on a series of spectra varying in S/N ratio. 

In [2]:
%matplotlib widget

In [3]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:75% !important; }</style>"))
display(HTML("<style>.output_result { max-width:75% !important; }</style>"))

In [4]:
import numpy as np
from numpy.random import RandomState
from time import time 
from tqdm.notebook import tqdm
import multiprocessing

from astropy.io import fits

from ppxftests.run_ppxf import run_ppxf
from ppxftests.ssputils import load_ssp_templates
from ppxftests.mockspec import create_mock_spectrum, calculate_mw_age
from ppxftests.ppxf_plot import plot_sfh_mass_weighted

import matplotlib.pyplot as plt
plt.ion()
plt.close("all")

from IPython.core.debugger import Tracer

In [4]:
###########################################################################
# Settings
###########################################################################
isochrones = "Padova"
sigma_star_kms = 250
z = 0.01

In [ ]:
###########################################################################
# Generate the input SFH
###########################################################################
# Load the stellar templates so we can get the age & metallicity dimensions
_, _, metallicities, ages = load_ssp_templates(isochrones)
N_ages = len(ages)
N_metallicities = len(metallicities)

# Simple Gaussian SFH
xx, yy = np.meshgrid(range(N_ages), range(N_metallicities))
x0 = 15
y0 = 2.5
sigma_x = 4
sigma_y = 0.1
sfh_young = np.exp(- (xx - x0)**2 / (2 * sigma_x**2)) *\
               np.exp(- (yy - y0)**2 / (2 * sigma_y**2))
sfh_young /= np.nansum(sfh_young)
sfh_mw_young = sfh_young * 1e8

x0 = 60
y0 = 4
sigma_x = 5
sigma_y = 0.5
sfh_old = np.exp(- (xx - x0)**2 / (2 * sigma_x**2)) *\
             np.exp(- (yy - y0)**2 / (2 * sigma_y**2))
sfh_old /= np.nansum(sfh_old)
sfh_mw_old = sfh_old * 1e10

# Add the young & old components
sfh_mw_original = sfh_mw_old + sfh_mw_young

# Instead of a Gaussian SFH, try with a "delta function" SFH (i.e., 1 old & 1 young template)
sfh_mw_original = np.zeros((N_metallicities, N_ages))
sfh_mw_original[2, 4] = 0.5e7
sfh_mw_original[1, 4] = 1.5e7
sfh_mw_original[0, 20] = 1e8
sfh_mw_original[1, -3] = 1e10

sfh_mw_1D_original = np.nansum(sfh_mw_original, axis=0)

# Compute the mean mass-weighted age below 1 Gyr
log_age_mw_original, log_age_mw_original_idx = calculate_mw_age(sfh_mw_original, age_thresh=1e9, ages=ages)


In [11]:
###########################################################################
# Check the spectrum and SFH
###########################################################################
spec_original, spec_original_err, lambda_vals_A = create_mock_spectrum(
    sfh_mass_weighted=sfh_mw_original,
    isochrones=isochrones, z=z, SNR=100, sigma_star_kms=sigma_star_kms,
    plotit=True)  


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/home/u5708159/python/Modules/ppxftests/mockspec.py:366: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(fig_w, fig_h))


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

No handles with labels found to put in legend.
/home/u5708159/python/Modules/ppxftests/ppxf_plot.py:30: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(10, 3.5))


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## S/N TESTING
---
In each iteration, run ppxf on a spectrum with a fixed input SFH but varying S/N.

In [12]:
SNR_vals = [5, 10, 20, 50, 100, 1000, 10000]
log_age_mw_regul_list = []
log_age_mw_MC_list = []
log_age_mw_MC_err_list = []

# For the MC simulation
niters = 20
nthreads = 20

for SNR in tqdm(SNR_vals):
    ###########################################################################
    # Generate the spectrum
    ###########################################################################
    spec_original, spec_original_err, lambda_vals_A = create_mock_spectrum(
        sfh_mass_weighted=sfh_mw_original,
        isochrones=isochrones, z=z, SNR=SNR, sigma_star_kms=sigma_star_kms,
        plotit=False)  

    ###########################################################################
    # Run ppxf WITH regularisation
    ###########################################################################
    t = time()
    pp_regul = run_ppxf(spec=spec_original, spec_err=spec_original_err, lambda_vals_A=lambda_vals_A,
                        z=z, ngascomponents=1,
                        regularisation_method="auto",
                        isochrones="Padova",
                        fit_gas=False, tie_balmer=True,
                        delta_regul_min=1, regul_max=5e4, delta_delta_chi2_min=50,
                        plotit=False, savefigs=False, interactive_mode=False)
    print(f"Total time in run_ppxf: {time() - t:.2f} seconds")

    ###########################################################################
    # Run ppxf WITHOUT regularisation, using a MC approach
    ###########################################################################
    # Helper function for multiprocessing
    def ppxf_helper(seed):
        # Add "extra" noise to the spectrum
        rng = RandomState(seed)
        noise = rng.normal(scale=spec_original_err)
        spec = spec_original + noise

        # This is to mitigate the "edge effects" of the convolution with the LSF
        spec[0] = -9999
        spec[-1] = -9999

        # Run ppxf
        pp = run_ppxf(spec=spec, spec_err=spec_original_err, lambda_vals_A=lambda_vals_A,
                      z=z, ngascomponents=1,
                      regularisation_method="none", 
                      isochrones="Padova",
                      fit_gas=False, tie_balmer=True,
                      plotit=False, savefigs=False, interactive_mode=False)
        return pp

    # Input arguments
    args_list = list(np.random.randint(low=0, high=100 * niters, size=niters))

    # Run in parallel
    print(f"Running ppxf on {nthreads} threads...")
    t = time()
    with multiprocessing.Pool(nthreads) as pool:
        pp_list = list(tqdm(pool.imap(ppxf_helper, args_list), total=niters))
    print(f"Elapsed time in ppxf: {time() - t:.2f} s")

    ###########################################################################
    # Compute the mass-weighted age 
    ###########################################################################
    log_age_mw_regul, log_age_mw_regul_idx = calculate_mw_age(pp_regul.weights_mass_weighted, age_thresh=1e9, ages=ages)

    log_age_mw_list = []
    for pp in pp_list:
        log_age_mw, _ = calculate_mw_age(pp.weights_mass_weighted, age_thresh=1e9, ages=ages)
        log_age_mw_list.append(log_age_mw)

    log_age_mw_MC = np.nanmean(log_age_mw_list)
    log_age_mw_MC_err = np.nanstd(log_age_mw_list)
    # For plotting purposes, figure out the approx. index of these ages in the age array
    log_age_mw_MC_idx = (log_age_mw_MC - np.log10(ages[0])) / (np.log10(ages[1]) - np.log10(ages[0]))
    log_age_mw_MC_err_idx = log_age_mw_MC_err / (np.log10(ages[1]) - np.log10(ages[0]))

    print("------------------------------------------------------------")
    print(f"SNR = {SNR:d}:")
    print(f"MC method:    mean mass-weighted age (log yr) = {np.nanmean(log_age_mw_list):.2f} ± {np.nanstd(log_age_mw_list):.2f}")
    print(f"Regul method: mean mass-weighted age (log yr) = {log_age_mw_regul:.2f}")
    print(f"Input value:  mean mass-weighted age (log yr) = {log_age_mw_original:.2f})")
    print("------------------------------------------------------------")
    
    # Append to list
    log_age_mw_regul_list.append(log_age_mw_regul)
    log_age_mw_MC_list.append(log_age_mw_MC)
    log_age_mw_MC_err_list.append(log_age_mw_MC_err)

    ###########################################################################
    # COMPARE THE INPUT AND OUTPUT
    ###########################################################################
    sfh_fit_mw_list = []
    sfh_fit_lw_list = []
    sfh_fit_mw_1D_list = []
    sfh_fit_lw_1D_list = []

    for pp in pp_list:
        sfh_fit_mw_list.append(pp.weights_mass_weighted)
        sfh_fit_lw_list.append(pp.weights_light_weighted)
        sfh_fit_mw_1D_list.append(np.nansum(pp.weights_mass_weighted, axis=0))
        sfh_fit_lw_1D_list.append(np.nansum(pp.weights_light_weighted, axis=0))

    # Compute the mean SFH 
    sfh_fit_mw_mean = np.nansum(np.array(sfh_fit_mw_list), axis=0) / len(sfh_fit_mw_list)
    sfh_fit_lw_mean = np.nansum(np.array(sfh_fit_lw_list), axis=0) / len(sfh_fit_lw_list)
    sfh_fit_mw_1D_mean = np.nansum(sfh_fit_mw_mean, axis=0)
    sfh_fit_lw_1D_mean = np.nansum(sfh_fit_lw_mean, axis=0)

    sfh_fit_mw_1D_regul = np.nansum(pp_regul.weights_mass_weighted, axis=0)
    sfh_fit_lw_1D_regul = np.nansum(pp_regul.weights_light_weighted, axis=0)

    # Plot the mass-weighted weights, summed over the metallicity dimension
    for log_scale in [True, False]:
        # Create new figure 
        fig = plt.figure(figsize=(13, 4))
        ax = fig.add_axes([0.1, 0.2, 0.7, 0.7])
        ax.set_title(f"Mass-weighted template weights (SNR = {SNR:d})")

        # Plot the SFHs from each ppxf run, plus the "truth" SFH
        ax.fill_between(range(N_ages), sfh_mw_1D_original, step="mid", alpha=1.0, color="cornflowerblue", label="Input SFH")
        for jj in range(niters):
            ax.step(range(N_ages), sfh_fit_mw_1D_list[jj], color="pink", alpha=0.2, where="mid", linewidth=0.25, label="ppxf fits (MC simluations)" if jj == 0 else None)
        ax.step(range(N_ages), sfh_fit_mw_1D_mean, color="red", where="mid", label="Mean ppxf fit (MC simulations)")
        ax.step(range(N_ages), sfh_fit_mw_1D_regul, color="lightgreen", where="mid", label="ppxf fit (regularised)")

        # Plot horizontal error bars indicating the mean mass-weighted age from (a) the MC simulations and (b) the regularised fit 
        y = 10**(0.9 * np.log10(ax.get_ylim()[1])) if log_scale else 0.9 * ax.get_ylim()[1]
        ax.errorbar(x=log_age_mw_regul_idx, y=y, xerr=0, yerr=0, 
                    marker="D", mfc="lightgreen",mec="lightgreen",  ecolor="lightgreen",  
                    label="MW age ($< 1$ Gyr) (regularised fit)")
        ax.errorbar(x=log_age_mw_MC_idx, y=y, xerr=log_age_mw_MC_err_idx, yerr=0, 
                    marker="D", mfc="red", mec="red", ecolor="red",
                    label="MW age ($< 1$ Gyr) (MC simulations)")
        ax.errorbar(x=log_age_mw_original_idx, y=y, xerr=0, yerr=0, 
                    marker="D", mfc="cornflowerblue", mec="cornflowerblue", ecolor="cornflowerblue", 
                    label="MW age ($< 1$ Gyr) (input)")

        # Decorations 
        ax.set_xticks(range(N_ages))
        ax.set_xlabel("Age (Myr)")
        ax.set_xticklabels(["{:}".format(age / 1e6) for age in ages], rotation="vertical", fontsize="x-small")
        ax.autoscale(axis="x", enable=True, tight=True)
        ax.set_ylim([1, None])
        ax.set_ylabel(r"Template weight ($\rm M_\odot$)")
        ax.legend(fontsize="x-small", loc="center left", bbox_to_anchor=(1.01, 0.5))
        ax.set_xlabel("Age (Myr)")
        ax.set_yscale("log") if log_scale else None
        

----------------------------------------------------
Iteration 0: Elapsed time in PPXF (single thread): 2.72 s
----------------------------------------------------
Iteration 1: Scaling noise by 1.0050...
Iteration 1: Running ppxf on 20 threads...
Iteration 1: Elapsed time in PPXF (multithreaded): 46.17 s
Iteration 1: optimal regul = 10000.00; Δm = 1.4072e+10; Δregul = 500.00 (Δregul_min = 1.00); Δχ (goal) - Δχ = 79.016
----------------------------------------------------
Iteration 2: Re-running ppxf on 20 threads (iteration 2)...
Iteration 2: Elapsed time in PPXF (multithreaded): 49.60 s
Iteration 2: optimal regul = 20000.00; Δm = 2.63388e+08; Δregul = 500.00 (Δregul_min = 1.00); Δχ (goal) - Δχ = 77.280
----------------------------------------------------
Iteration 3: Re-running ppxf on 20 threads (iteration 3)...
Iteration 3: Elapsed time in PPXF (multithreaded): 49.29 s
Iteration 3: optimal regul = 30000.00; Δm = 2.31653e+08; Δregul = 500.00 (Δregul_min = 1.00); Δχ (goal) - Δχ = 75.4


Elapsed time in ppxf: 31.05 s
------------------------------------------------------------
SNR = 5:
MC method:    mean mass-weighted age (log yr) = 7.73 ± 0.23
Regul method: mean mass-weighted age (log yr) = 8.27
Input value:  mean mass-weighted age (log yr) = 8.52)
------------------------------------------------------------


/pkg/linux/anaconda-20191122/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:119: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

...

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

----------------------------------------------------
Iteration 0: Elapsed time in PPXF (single thread): 3.35 s
----------------------------------------------------
Iteration 1: Scaling noise by 98.1077...
Iteration 1: Running ppxf on 20 threads...
Iteration 1: Elapsed time in PPXF (multithreaded): 68.01 s
Iteration 1: optimal regul = 0.00; Δm = 0.417245; Δregul = 500.00 (Δregul_min = 1.00); Δχ (goal) - Δχ = 95.079
----------------------------------------------------
Iteration 2: Re-running ppxf on 20 threads (iteration 2)...
Iteration 2: Elapsed time in PPXF (multithreaded): 56.19 s
Iteration 2: optimal regul = 100.00; Δm = 1.24343e+10; Δregul = 100.00 (Δregul_min = 1.00); Δχ (goal) - Δχ = 65.865
----------------------------------------------------
Iteration 3: Re-running ppxf on 20 threads (iteration 3)...
Iteration 3: Elapsed time in PPXF (multithreaded): 59.54 s
Iteration 3: optimal regul = 40.00; Δm = 2.96689e+09; Δregul = 20.00 (Δregul_min = 1.00); Δχ (goal) - Δχ = 8.709
---------


Elapsed time in ppxf: 37.42 s
------------------------------------------------------------
SNR = 10000:
MC method:    mean mass-weighted age (log yr) = 7.43 ± 0.00
Regul method: mean mass-weighted age (log yr) = 7.51
Input value:  mean mass-weighted age (log yr) = 8.52)
------------------------------------------------------------


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
log_age_mw_original

In [ ]:
###########################################################################
# Plot: mass-weighted mean age as a function of SNR
###########################################################################
fig, ax = plt.subplots(nrows=1, ncols=1)
ax.axhline(log_age_mw_original, color="black", label="Input")
ax.errorbar(x=SNR_vals, y=log_age_mw_MC_list, yerr=log_age_mw_MC_err_list, marker="D", label="MC simulations")
ax.errorbar(x=SNR_vals, y=log_age_mw_regul_list, marker="D", label="Regularisation")
ax.set_xlabel("SNR")
ax.set_ylabel("Mass-weighted mean age (<1Gyr)")
ax.grid()
